## Рекомендательные системы

Задача рекомендаций музыки. Реализация метода коллаборативной фильтрации и модели со скрытыми переменными.

In [62]:
from sklearn.preprocessing import LabelEncoder

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from typing import Callable, List

import matplotlib.pyplot as plt
import seaborn as sns
import scipy.sparse as scs

#### Реализация метрики $MAP@k$ для оценки качества рекомендаций

$$
MAP@k = \frac{1}{N} \sum_{u = 1}^N AP_u@k
$$
$$
AP_u@k = \frac{1}{\min(k, n_u)} \sum_{i=1}^k r_u(i) p_u@i
$$
$$p_u@k = \dfrac{1}{k}\sum_{j=1}^k r_u(j)$$


*   $N$ - количество пользователей.
*   $n_u$ - число релевантных треков пользователя $u$ на тестовом промежутке.
*   $r_u(i)$ - бинарная величина: относится ли трек на позиции $i$ к релевантным.

In [63]:
def mapk(relevant: List[List[int]], predicted: List[List[int]], k: int = 20):
    map_k, cnt = 0, 0
    for relevant_items, predicted_items in zip(relevant, predicted):
      if not relevant_items:
        continue

      correct, score = 0, 0
      predicted_items = predicted_items[:k]
      for i, pred in enumerate(predicted_items):
        if pred in relevant_items and pred not in predicted_items[:i]:
          correct += 1
          score += correct / (i + 1)
      apk = score / min(len(relevant_items), k)

      map_k += apk
      cnt += 1
    return map_k / cnt

In [64]:
relevant = [
    [1, 7, 6, 2, 8],
    [1, 5, 4, 8],
    [8, 2, 5]
]

pred = [
    [8, 1, 5, 0, 7, 2, 9, 4],
    [0, 1, 8, 5, 3, 4, 7, 9],
    [9, 2, 0, 6, 8, 5, 3, 7]
]

assert round(mapk(relevant, pred, k=5), 4) == 0.4331

#### Данные

In [65]:
ratings = pd.read_csv('11_recsys_data.csv')
ratings.head()

,userId,trackId
0,0,14
1,0,95
2,0,219
3,0,220
4,0,404


In [66]:
tracks_info = pd.read_csv('11_recsys_data_1.csv')
tracks_info.head()

,id,name,artists
0,0,What There Is,['a-ha']
1,1,I'll Play The Blues For You,['Albert King']
2,2,Breaking Up Somebody's Home,['Albert King']
3,3,Imma Be,['Black Eyed Peas']
4,4,Boom Boom Pow,['Black Eyed Peas']


Разделим данные на тренировочные и тестовые так, чтобы в теcтовый датасет попали 50 последних треков каждого пользователя.

In [67]:
def train_test_split(ratings):
    train_ratings, test_ratings = [], []
    num_test_samples = 50

    # getting train samples
    for userId, user_data in tqdm(ratings.groupby('userId')):
        train_ratings += [user_data[:-num_test_samples]]

    train_ratings = pd.concat(train_ratings).reset_index(drop=True)
    all_train_items = train_ratings['trackId'].unique()

    # getting train samples
    # we drop all tracks that are not presented it the training samples,
    # because we won't be able to learn representations for them
    for userId, user_data in tqdm(ratings.groupby('userId')):
        test_items = user_data[-num_test_samples:]
        test_items = test_items[np.isin(test_items['trackId'], all_train_items)]
        test_ratings += [test_items]

    test_ratings = pd.concat(test_ratings).reset_index(drop=True)

    return train_ratings, test_ratings

In [68]:
train_ratings, test_ratings = train_test_split(ratings)

  0%|          | 0/241 [00:00<?, ?it/s]

  0%|          | 0/241 [00:00<?, ?it/s]

Почистим табличку с информацией о треках и закодируем id треков так, чтобы они соответствовали их порядковому номеру.

In [69]:
redundant_rows = np.where(~np.isin(tracks_info['id'], train_ratings['trackId'].unique()))[0]
tracks_info.drop(redundant_rows, inplace=True)
tracks_info = tracks_info.reset_index(drop=True)

In [70]:
def ids_encoder(ratings):
    users = sorted(ratings['userId'].unique())
    items = sorted(ratings['trackId'].unique())

    # create users and items encoders
    uencoder = LabelEncoder()
    iencoder = LabelEncoder()

    # fit users and items ids to the corresponding encoder
    uencoder.fit(users)
    iencoder.fit(items)

    return uencoder, iencoder

In [71]:
uencoder, iencoder = ids_encoder(train_ratings)
train_ratings['trackId'] = iencoder.transform(train_ratings['trackId'].tolist())
test_ratings['trackId'] = iencoder.transform(test_ratings['trackId'].tolist())
tracks_info['id'] = iencoder.transform(tracks_info['id'].tolist())

In [72]:
train_ratings.head()

,userId,trackId
0,0,14
1,0,95
2,0,219
3,0,220
4,0,404


In [73]:
test_ratings.head()

,userId,trackId
0,0,57582
1,0,57802
2,0,57957
3,0,58174
4,0,59168


Соберем все релевантные треки для каждого пользователя в список.

In [74]:
test_relevant = []
test_users = []
for user_id, user_data in test_ratings.groupby('userId'):
    test_relevant += [user_data['trackId'].tolist()]
    test_users.append(user_id)

#### Реализация класса `BaseModel`

In [75]:
class BaseModel:
    def __init__(self, ratings: pd.DataFrame):
        self.ratings = ratings
        self.n_users = len(np.unique(self.ratings['userId']))
        self.n_items = len(np.unique(self.ratings['trackId']))

        self.R = np.zeros((self.n_users, self.n_items))
        self.R[self.ratings['userId'], self.ratings['trackId']] = 1.

    def recommend(self, uid: int):
        """
        param uid: int - user's id
        return: [n_items] - vector of recommended items sorted by their scores in descending order
        """
        raise NotImplementedError

    def remove_train_items(self, preds: List[List[int]], k: int):
        """
        param preds: [n_users, n_items] - recommended items for each user
        param k: int
        return: np.array [n_users, k] - recommended items without training examples
        """
        new_preds = np.zeros((len(preds), k), dtype=int)
        for user_id, user_data in self.ratings.groupby('userId'):
            user_preds = preds[user_id]
            new_preds[user_id] = user_preds[~np.in1d(user_preds, user_data['trackId'])][:k]

        return new_preds

    def get_test_recommendations(self, k: int): # метод принимает на вход параметр `k` и возвращает массив из `k` наиболее подходящих треков для каждого пользователя
        test_preds = []

        for user_id, user_data in self.ratings.groupby('userId'):
            user_preds = self.recommend(user_id)
            test_preds.append(user_preds)
        test_preds = self.remove_train_items(test_preds, k) # удаление уже прослушанных треков из рекомендуемых

        return test_preds[test_users]

#### Коллаборативная фильтрация (User2User)

Идея: чтобы выбрать треки, которые понравятся пользователю, можно набрать несколько похожих на него пользователей (соседей) и посмотреть, какие треки они слушают. После этого остается агрегировать треки этих пользователей и выбрать самые популярные. Соответственно, задача состоит из двух частей: выбора функции похожести двух пользователей и способа агрегации.

В качестве функции похожести используются две метрики:

1. Корреляция Пирсона (видоизменена, чтобы подходить под задачу)

$$s(u, v) = \frac{\sum_{i \in I_u \cap I_v} r_{ui}r_{vi}}{\sqrt{\sum_{i \in I_u} r_{ui} ^2}\sqrt{\sum_{i \in I_v} r_{vi}^2}} $$

2. Мера Жаккара

$$ s(u, v) = \frac{|I_u \cap I_v|}{|I_u \cup I_v|} $$

Во всех формулах
* $I_u$ - множество треков, прослушанных пользователем $u$.
* $r_{ui}$ - прослушал ли пользователь $u$ трек $i$ (0 или 1).

Множество соседей определим как $$N(u) = \{ v \in U \setminus \{u\} \mid s(u, v) > \alpha\},$$ где $\alpha \, - $ гиперпараметр.

Для агрегации используется следующая формула:
$$
\hat{r}_{ui} = \frac{\sum_{v \in N(u)} s(u, v) r_{vi}}{\sum_{v \in N(u)} |s(u, v)|}
$$

**Реализация функций подсчета корреляции Пирсона и меры Жаккара**

Функции принимают матрицу оценок и вектор оценок пользователя $u$ и возвращают вектор со значениями похожести пользователя $u$ на всех пользователей.

In [76]:
def pearson(ratings: np.array, user_vector: np.array) -> np.array:
    dot_product = np.dot(ratings, user_vector)
    denominator = np.linalg.norm(user_vector) * np.linalg.norm(ratings, axis=1)
    return dot_product / denominator


def jaccard(ratings: np.array, user_vector: np.array) -> np.array:
    intersection = np.sum(np.logical_and(ratings, user_vector), axis=1)
    union = np.sum(np.logical_or(ratings, user_vector), axis=1)
    return intersection / union

In [77]:
ratings = [[0, 1, 1, 0, 1],
           [1, 0, 1, 1, 1]]
user_vector = [0, 1, 0, 1, 0]
pearson(ratings, user_vector)

array([0.40824829, 0.35355339])

In [78]:
jaccard(ratings, user_vector)

array([0.25, 0.2 ])

**Реализация класса `User2User`**

In [79]:
class User2User(BaseModel):
    def __init__(self, ratings, similarity_func):
        super().__init__(ratings)

        assert similarity_func in [pearson, jaccard]

        self.similarity_func = similarity_func
        self.alpha = 0.02

    def similarity(self, user_vector: np.array):
        """
        user_vector: [n_items]
        """
        return np.where(self.similarity_func(self.ratings, user_vector) > self.alpha, self.similarity_func(self.ratings, user_vector), 0)

    def recommend(self, uid: int): # метод возвращает индексы треков, отсортированные в порядке убывания предсказанных оценок
        similarities = self.similarity(self.ratings[uid])
        preds = np.dot(similarities.T, self.ratings) / np.sum(np.abs(similarities))
        return np.argsort(preds)[::-1]

In [80]:
ratings = [[0, 1, 1, 0, 1],
           [1, 0, 1, 1, 1]]
user_vector = [0, 1, 0, 1, 0]
similarities = np.where(pearson(ratings, user_vector) > 0.02, pearson(ratings, user_vector), 0)
print(similarities)
preds = np.dot(similarities.T, ratings) / np.sum(np.abs(similarities))
print(preds)
np.argsort(preds)[::-1]

[0.40824829 0.35355339]
[0.46410162 0.53589838 1.         0.46410162 1.        ]


array([4, 2, 1, 3, 0])

Посмотрим, насколько хорошо модель рекомендует треки. Для этого сравним уже прослушанные треки с рекомендованными и релевантными для случайного пользователя.

In [81]:
model = User2User(ratings=train_ratings, similarity_func=jaccard)
user_id = np.random.randint(0, model.n_users)

In [ ]:
listened_tracks = train_ratings[train_ratings.userId == user_id].trackId[:15]

print('Already listened tracks:')

tracks_info.loc[listened_tracks][['name', 'artists']]

Already listened tracks:


,name,artists
300,All Star,['Smash Mouth']
831,Tokyo Drift (Fast & Furious),['Teriyaki Boyz']
7272,Around the World (La La La La La),['A Touch Of Class']
8119,Still D.R.E.,['Dr. Dre']
9366,The X-Files,['The X Project']
10707,End Credits,['Hans Zimmer']
16623,Dum Dee Dum,['Keys N Krates']
16911,Mi Mi Mi,['Selene RMX']
18237,Turn Down for What,"['DJ Snake', 'Lil Jon']"
19333,Turn Down for What,"['DJ Snake', 'Lil Jon', 'Juicy J', '2 Chainz',..."


In [ ]:
preds = model.get_test_recommendations(15)

print('Predicted tracks:')

tracks_info.loc[preds[user_id]][['name', 'artists']]

  0%|          | 0/241 [00:00<?, ?it/s]

Predicted tracks:


,name,artists
2814,Numb,['Linkin Park']
24500,Way Down We Go,['KALEO']
1073,Smells Like Teen Spirit,['Nirvana']
805,Zombie,['The Cranberries']
1019,It's My Life,['Bon Jovi']
11493,The Show Must Go On,['Queen']
7533,Highway to Hell,['AC/DC']
49577,Кукла колдуна,['Король и Шут']
18459,Take Me To Church,['Hozier']
8263,Shape Of My Heart,['Sting']


In [ ]:
test_tracks = test_ratings[test_ratings.userId == user_id].trackId[:15]

print('Test-time tracks:')

tracks_info.loc[test_tracks][['name', 'artists']]

Test-time tracks:


,name,artists
65569,Little Do You Know Beat Cry,['Yagih Mael']
65897,Life Goes On,['Oliver Tree']
65904,Gdzie jest biały węgorz ? (Zejście),['Cypis']
65918,Him & Her,['Pacino']
65948,I WANT YOU BACK,['Dazel Ukuto']
65967,Levan Polka,"['Dance', 'Tendência', 'Baila']"
66260,Levan Polkka,['Dj Mix Urbano']
66261,Kulikitaka Challenge,['Dj Mix Urbano']
66299,In Da Getto,"['J. Balvin', 'Skrillex']"
66314,Entrenamiento en el Gym,['Gimnasio de motivación']


#### Модель со скрытыми переменными

Идея: будем предсказывать оценки по формуле
$$
\hat{r}_{ui} = \langle p_u, q_u \rangle,
$$
$p_u \in \mathbb{R}^d$ и $q_i \in \mathbb{R}^d$ - латентные векторы пользователя $u$ и объекта $i$ соответственно.

Оптимизировать будем MSE между истинной оценкой пользователя и предсказанной с регуляризацией:
$$
L = \sum_{(u, i) \in R} (\hat{r}_{ui} - r_{ui})^2 + \lambda \left(\sum_{u \in U} \|p_u\|^2 + \sum_{i \in I} \|q_i\|^2\right)
$$

Рассмотрим два подхода к оптимизации параметров: можно это делать обычным стохастическим градиентным спуском, а можно по очереди обновлять матрицы $P, Q$ (метод Alternating Least Squares, ALS). Формулы обновления параметров для обоих методов приведены ниже.

LFM:

$$
\left\{
    \begin{array}\\
        p_u \leftarrow p_u + \eta\cdot(e_{ui}q_i - \lambda p_u)
        \\q_i \leftarrow q_i + \eta\cdot(e_{ui}p_u - \lambda q_i)\\
    \end{array}
\right.
, e_{ui} = r_{ui}-\hat{r}_{ui}
$$

\\
ALS:

$$
\left\{
    \begin{array}\\
        p_u = (Q^T\cdot C^u \cdot Q + \lambda \cdot I)^{-1} \cdot Q^T \cdot C^u \cdot p(u)
        \\q_i = (P^T\cdot C^i \cdot P + \lambda \cdot I)^{-1} \cdot P^T \cdot C^i \cdot p(i)\\
    \end{array}
\right.
$$

\\

**Реализация методов оптимизации параметров для алгоритмов LFM и ALS**

In [82]:
class HiddenVars(BaseModel):
    def __init__(self, ratings, dim=128, mode='sgd', alpha=40):
        super().__init__(ratings)
        self.dim = dim

        assert mode in ['sgd', 'als']
        self.mode = mode

        self.P = np.random.normal(size=(self.n_users, dim))
        self.Q = np.random.normal(size=(self.n_items, dim))

        self.lr = 0.0003
        self.lamb = 0.01
        self.alpha = alpha

    def fit(self, num_iters=5):
        for epoch in tqdm(range(num_iters)):

            if self.mode == 'sgd':
                for u, i, r in self.ratings:
                    err = r - np.dot(self.P[u], self.Q[i])
                    self.P[u] += self.lr * (err * self.Q[i] - self.lamb * self.P[u])
                    self.Q[i] += self.lr * (err * self.P[u] - self.lamb * self.Q[i])

            elif self.mode == 'als':
                P = np.copy(self.ratings)
                P[P > 0] = 1 # матрица индикаторов совершенного действия
                C = 1 + self.alpha * self.ratings # матрица уверенностей
                for u in range(self.n_users):
                    C_u = np.diag(C[u])
                    self.P[u] = np.linalg.inv(self.Q.T @ C_u @ self.Q + self.lamb * np.eye(self.dim)) @ self.Q.T @ C_u @ P[u]

                for i in range(self.n_items):
                    C_i = np.diag(C[:, i])
                    self.Q[i] = np.linalg.inv(self.P.T @ C_i @ self.P + self.lamb * np.eye(self.dim)) @ self.P.T @ C_i @ P[:, i]

    def recommend(self, uid):
        pred_rating = self.P[uid] @ self.Q.T

        return np.argsort(pred_rating)[::-1]

In [84]:
model = HiddenVars(ratings=train_ratings)

При оптимизации параметров марицы $Q$ похожим трекам должны соответствовать похожие векторы. Поэтому можем для любого трека найти наиболее близкие к нему в латентном пространстве и проверить степерь обученности модели вручную.

In [85]:
example_trackId = tracks_info[tracks_info.name == 'Выхода нет'].iloc[0].id

preds = model.Q @ model.Q[example_trackId]
preds = preds / np.sqrt((model.Q**2).sum(axis=1) + 1e-8)

track_idxs = preds.argsort()[::-1][:20]

In [ ]:
similar_tracks = tracks_info.loc[track_idxs][['name', 'artists']]
similar_tracks['similarity'] = preds[track_idxs] / np.linalg.norm(model.Q[example_trackId])
similar_tracks

,name,artists,similarity
5512,Выхода нет,['Сплин'],1.000000
5517,Варвара,['Би-2'],0.649796
17328,Я хочу быть с тобой,['Nautilus Pompilius'],0.646846
2058,Последний герой,['КИНО'],0.640997
5872,Я свободен,['Кипелов'],0.606749
2060,Хочу перемен,['КИНО'],0.603231
5515,Романс,['Сплин'],0.590318
24284,Как на войне,['Агата Кристи'],0.586219
4463,Holiday,['Green Day'],0.576644
2179,Восьмиклассница,['КИНО'],0.570639
